In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Running Vertex Pipelines using E2E Samples repository - Infrastructure Cleanup


<table align="left">

  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/teamdatatonic/vertex-pipelines-end-to-end-samples/blob/develop/docs/notebooks/workbench/03_infrastructure_cleanup.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This notebook shows you how to destroy infrastructure which was created in ([01_infrastructure_setup.ipynb](01_infrastructure_setup.ipynb)).

### Objective

In this tutorial, you learn how to destroy infrastructure using terraform.

The steps performed include:

* Destroy infrastructure using Terraform.

### Prerequisites

- [Google Cloud SDK (gcloud)](https://cloud.google.com/sdk/docs/quickstart)
- Make
- [Terraform](https://www.terraform.io)
- Infrastructure deployed ([01_infrastructure_setup.ipynb](01_infrastructure_setup.ipynb)).

## Change working directory

In [ ]:
%cd vertex-pipelines-end-to-end-samples/

## Before You Begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[my-project-id]"  # @param {type:"string"}
# Set the project id
! gcloud config set project {PROJECT_ID}

### Authenticate your Google Cloud account


As you're running a Jupyter environment on workbench, you are already authenticated.

### Environment Setup

In order to run make commands relevant environment variables need to be set. You can re-use `.env` file defined in [01_infrastructure_setup.ipynb](01_infrastructure_setup.ipynb).

Load environment variables using [Python-dotenv](https://pypi.org/project/python-dotenv/).

In [ ]:
! pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:


**Empty the buckets to enable their deletion:**

In [ ]:
%%bash
gsutil rm -a gs://${VERTEX_PROJECT_ID}-staging/**
gsutil rm -a gs://${VERTEX_PROJECT_ID}-pl-root/**

**Delete data transfer config:**

*If transfer config name was not set automatically you can find it via Google Cloud Console UI (BigQuery/Data Transfers)*.

In [ ]:
import google.api_core.exceptions
from google.cloud import bigquery_datatransfer

transfer_client = bigquery_datatransfer.DataTransferServiceClient()

transfer_config_name = "my-transfer-config" # Transfer Config Resource Name (projects/{project_number}/locations/{location}/transferConfigs/{transfer_config_id)
try:
    transfer_client.delete_transfer_config(name=transfer_config_name)
except google.api_core.exceptions.NotFound:
    print("Transfer config not found.")
else:
    print(f"Deleted transfer config: {transfer_config_name}")

**Delete BigQuery dataset for the Chicago Taxi data:**

In [ ]:
! bq --location=${VERTEX_LOCATION} rm -f -r --dataset "${VERTEX_PROJECT_ID}:chicago_taxi_trips" 

**Delete BigQuery dataset for data processing during the pipelines:**

In [ ]:
! bq --location=${VERTEX_LOCATION} rm -f -r --dataset "${VERTEX_PROJECT_ID}:preprocessing" 

**Destroy infrastructure by executing undeploy command (terraform destroy):**

In [ ]:
import os
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['HOME'], '.bin')
! make undeploy auto-approve=true

**Delete local files:**

In [ ]:
%%bash
cd ..
rm -r vertex-pipelines-end-to-end-samples

### Congratulations on successfully destroying your infrastructure!